In [1]:
# Adding NeurIPS to the path
import sys
import os

In [1]:
from ruben import PersistenceDiagram

In [2]:
import pickle
import pathlib
import math
from gtda.diagrams import PersistenceEntropy, NumberOfPoints, Amplitude, PersistenceImage, ComplexPolynomial, BettiCurve
from functools import partial 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn.utils import shuffle
import seaborn as sns
import random
# from model.Google.MutualInformationGoogle import get_mutual_information

from sklearn.metrics import r2_score

2022-12-17 20:11:56.780323: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Constants
def get_gen_gaps_folder(task_number):
    return os.path.abspath('../../NeurIPSSoftware/results/task{}/generalization_gap'.format(task_number))
def get_pds_folder(task_number):
    return os.path.abspath('../../out/task{}/pds'.format(task_number))

In [5]:
import json
from pathlib import Path

def load_generalization_gap(model_file, folder_generalization):
    with open('{}/{}'.format(folder_generalization, model_file), 'rb') as file:
        gen_gap = pickle.load(file)
    return gen_gap

def load_generalization_gaps(task_number):
    # folder_path = get_gen_gaps_folder(task_number)
    # model_names_w_ext = os.listdir(folder_path)
    # model_names = map(lambda name: name[:-4], model_names_w_ext)
    # gen_gaps = map(lambda model_name: load_generalization_gap(model_name, folder_path), model_names_w_ext)
    # return dict(zip(model_names, gen_gaps))
    ref_file = Path(
        "/Users/otis/Documents/rubens_speelhoekje/google/google_data/public_data/reference_data/task1_v4/model_configs.json")
    return {
        'model_' + k: v['metrics']['train_acc'] - v['metrics']['test_acc']
        for k, v in json.loads(ref_file.read_text()).items()
    }

In [6]:
from pathlib import Path

outdir = Path('../out/task1/random')

def load_pd(path):
    # with open('{}/{}'.format(folder_pd, model_file), 'rb') as file:
    #     pd = pickle.loads(file.read().replace(b'model.PersistenceDiagram',b'model.homology.PersistenceDiagram'))
    # return pd
    with open(path, 'rb') as f:
        # return pickle.loads(f.read().replace(b'model.PersistenceDiagram',b'model.homology.PersistenceDiagram'))
        return pickle.load(f)

def load_pds(task_number):
    # folder_path = get_pds_folder(task_number)
    # pds_tries = os.listdir(folder_path)
    # pds_by_model_number = dict()
    # for pd_try in pds_tries:
    #     try_path = '{}/{}'.format(folder_path, pd_try)
    #     model_names_w_ext = os.listdir(try_path)
    #     for model_name_w_ext in model_names_w_ext:
    #         model_name = model_name_w_ext[:-4]
    #         pd = load_pd(model_name_w_ext, try_path)
    #         if model_name not in pds_by_model_number:
    #             pds_by_model_number[model_name] = [pd]
    #         else:
    #             pds_by_model_number[model_name].append(pd)
    # return pds_by_model_number
    assert task_number == 1
    return {p.stem: [load_pd(p)] for p in outdir.glob('model_*.bin')}

def get_pds_and_generalization_gaps(task_number):
    pds = load_pds(task_number)
    gen_gaps = load_generalization_gaps(task_number)
    model_names = pds.keys()
    gen_gaps_and_pds = dict()
    for model_name in model_names:
        gen_gaps_and_pds[model_name] = (gen_gaps[model_name], pds[model_name])
    return list(gen_gaps_and_pds.items())

In [7]:
get_pds_and_generalization_gaps(1)

[('model_158',
  (0.2593799829483032, [<ruben.PersistenceDiagram at 0x138330070>])),
 ('model_602',
  (0.30239999294281006, [<ruben.PersistenceDiagram at 0x104051610>])),
 ('model_212',
  (0.25919997692108154, [<ruben.PersistenceDiagram at 0x11b846a60>])),
 ('model_213',
  (0.2612000107765198, [<ruben.PersistenceDiagram at 0x138330ac0>])),
 ('model_603',
  (0.3065999746322632, [<ruben.PersistenceDiagram at 0x1382f7580>])),
 ('model_159',
  (0.2656399607658386, [<ruben.PersistenceDiagram at 0x13833b2b0>])),
 ('model_601',
  (0.23843997716903687, [<ruben.PersistenceDiagram at 0x1382a38e0>])),
 ('model_600',
  (0.23054003715515137, [<ruben.PersistenceDiagram at 0x13835c040>])),
 ('model_604',
  (0.17900002002716064, [<ruben.PersistenceDiagram at 0x13816c190>])),
 ('model_599',
  (0.20836001634597778, [<ruben.PersistenceDiagram at 0x138162fd0>])),
 ('model_214',
  (0.2841799855232239, [<ruben.PersistenceDiagram at 0x1381eb700>])),
 ('model_215',
  (0.28255999088287354, [<ruben.PersistenceD

In [8]:
def get_TASK_1_pds_and_generalization_gaps():
    return get_pds_and_generalization_gaps(1)

In [9]:
def get_TASK_2_pds_and_generalization_gaps():
    return get_pds_and_generalization_gaps(2)

In [10]:
def split_train_and_test(gen_x_pds, train_size=0.7):
    training_elems = int(math.floor(train_size*len(gen_x_pds)))
    return gen_x_pds[:training_elems], gen_x_pds[training_elems:]

In [11]:
def shuffle_and_split_train_and_test(gen_x_pds):
    shuffled_gen_x_pds = shuffle(gen_x_pds)
    return split_train_and_test(shuffled_gen_x_pds)

In [12]:
pds_x_gen_task1 = get_TASK_1_pds_and_generalization_gaps()
# pds_x_gen_task2 = get_TASK_2_pds_and_generalization_gaps()

In [13]:
def classic_linear_model():
    return LinearRegression()
def ridge_linear_model():
    return linear_model.Ridge(alpha=.5)
def lasso_linear_model():
    return linear_model.Lasso(alpha=0.1)
def elasticNet():
    return ElasticNet()

# Techniques for extracting features

See the other notebooks to see detailed descriptions

In [14]:
from ruben import PersistenceDiagram
# from model.TopologicalSummaries.PersistenceDiagramSummaries import average_life, average_midlife
class AllDims:
    def __contains__(self, _): return True


all_dims = AllDims() # cannot be static method

def is_collection(x):
    return hasattr(x,"__contains__")

def alpha_weighted_average_life(persistence_diagram: PersistenceDiagram, alpha: float, dimensions=all_dims):
    if not is_collection(dimensions): dimensions = [dimensions]
    points = filter(lambda point: point.dim in dimensions, persistence_diagram.points)

    life = lambda point: point.death - point.birth
    values = map(lambda point: life(point)*life(point)**alpha, points)

    val_without_inf = filter(lambda val: not np.isinf(val), values)
    return np.mean(np.fromiter(val_without_inf, dtype=np.double))

def alpha_weighted_average_midlife(persistence_diagram: PersistenceDiagram, alpha: float, dimensions=all_dims):
    if not is_collection(dimensions): dimensions = [dimensions]
    points = filter(lambda point: point.dim in dimensions, persistence_diagram.points)

    life = lambda point: point.death - point.birth
    midlife = lambda point: (point.death + point.birth)/2
    values = map(lambda point: midlife(point)*life(point)**alpha, points)

    val_without_inf = filter(lambda val: not np.isinf(val), values)
    return np.mean(np.fromiter(val_without_inf, dtype=np.double))

def average_life(persistence_diagram: PersistenceDiagram, dimensions=all_dims):
    return alpha_weighted_average_life(persistence_diagram, 0, dimensions=dimensions)

def average_midlife(persistence_diagram: PersistenceDiagram, dimensions=all_dims):
    return alpha_weighted_average_midlife(persistence_diagram, 0, dimensions=dimensions)

In [15]:
def get_pooling_vector(persistence_diagram, dims, num_elems=10):
        finite_points_pd = persistence_diagram.get_finite_points()
        pooling_vector = []
        for dim in dims:
            points_of_dim = filter(lambda point: point.dim == dim, finite_points_pd)
            persistences = list(map(lambda point: point.death - point.birth, points_of_dim))
            persistences.sort(reverse=True)
            pooling_vector.extend(persistences[:num_elems])
            #If we don't have enough persistences we fill the vector with 0s
            if(len(persistences) < num_elems):
                pooling_vector.extend([0]*(num_elems - len(persistences)))
        return np.array(pooling_vector)

In [16]:
def get_life_midlife_vector(persistence_diagram, dims):
    lifes_midlifes = []
    for dim in dims:
        lifes_midlifes.append(average_life(persistence_diagram, dimensions=[dim]))
        lifes_midlifes.append(average_midlife(persistence_diagram, dimensions=[dim]))
    lifes_midlifes.append(average_life(persistence_diagram))
    lifes_midlifes.append(average_midlife(persistence_diagram))
    return np.nan_to_num(np.array(lifes_midlifes))

In [17]:
def get_life_midlife_vector_squared(persistence_diagram, dims):
    lifes_midlifes = []
    for dim in dims:
        lifes_midlifes.append(average_life(persistence_diagram, dimensions=[dim]))
        lifes_midlifes.append(average_midlife(persistence_diagram, dimensions=[dim]))
    lifes_midlifes.append(average_life(persistence_diagram))
    lifes_midlifes.append(average_midlife(persistence_diagram))
    life_midlife_array = np.nan_to_num(np.array(lifes_midlifes))
    return np.hstack([life_midlife_array, life_midlife_array**2])

In [18]:
def average_birth_death(points):
    birth_death_pairs = np.array(list(map(lambda point: [point.birth, point.death], points)))
    if birth_death_pairs.shape[0] == 0:
        return [0, 0]
    return np.average(birth_death_pairs, axis=0).tolist()

In [19]:
def get_average_birth_death_vector(persistence_diagram, dims):
    birth_eath_averages = []
    finite_points_pd = persistence_diagram.get_finite_points()
    for dim in dims:
        points_of_dim = list(filter(lambda point: point.dim == dim, finite_points_pd))
        birth_eath_averages.extend(average_birth_death(points_of_dim))
    birth_eath_averages.extend(average_birth_death(points_of_dim))
    return np.array(birth_eath_averages)

In [20]:
def get_average_birth_death_vector_squared(persistence_diagram, dims):
    birth_eath_averages = []
    finite_points_pd = persistence_diagram.get_finite_points()
    for dim in dims:
        points_of_dim = list(filter(lambda point: point.dim == dim, finite_points_pd))
        birth_eath_averages.extend(average_birth_death(points_of_dim))
    birth_eath_averages.extend(average_birth_death(points_of_dim))
    birth_death_vector = np.array(birth_eath_averages)
    return np.hstack([birth_death_vector, birth_death_vector**2])

In [21]:
def get_average_birth_death_vector_inverted(persistence_diagram, dims):
    birth_eath_averages = []
    finite_points_pd = persistence_diagram.get_finite_points()
    for dim in dims:
        points_of_dim = list(filter(lambda point: point.dim == dim, finite_points_pd))
        birth_eath_averages.extend(average_birth_death(points_of_dim))
    birth_eath_averages.extend(average_birth_death(points_of_dim))
    birth_death = np.array(birth_eath_averages)
    return np.hstack([np.log(birth_death+1), birth_death, np.nan_to_num(np.reciprocal(birth_death, out=np.zeros_like(birth_death), where=birth_death!=0))])

In [22]:
def get_average_birth_death_life_midlife_vector(persistence_diagram, dims):
    birth_death = get_average_birth_death_vector(persistence_diagram, dims)
    life_midlife = get_life_midlife_vector(persistence_diagram, dims)
    return np.hstack([birth_death**2, life_midlife**2, birth_death, life_midlife])

In [23]:
def get_entropy(persistence_diagram, dims):
    points = persistence_diagram.get_finite_points()
    pe = PersistenceEntropy()
    pe_vector = []
    for dim in dims:
        dim_points = filter(lambda point: point.dim == dim, points)
        points_3d = np.array([list(map(lambda point: [point.birth, point.death, point.dim], dim_points))])
        if(points_3d.shape[1] == 0):
            pe_vector.append(0)
        else:
            pe_vector.append(pe.fit_transform(points_3d)[0][0])
    return np.array(pe_vector)

In [24]:
def get_number_of_points(persistence_diagram, dims):
    points = persistence_diagram.get_finite_points()
    numPoints = NumberOfPoints()
    numPoints_vector = []
    for dim in dims:
        dim_points = filter(lambda point: point.dim == dim, points)
        points_3d = np.array([list(map(lambda point: [point.birth, point.death, point.dim], dim_points))])
        if(points_3d.shape[1] == 0):
            numPoints_vector.append(0)
        else:
            numPoints_vector.append(numPoints.fit_transform(points_3d)[0][0])
    return np.array(numPoints_vector)

In [25]:
def get_persistence_image_vector(persistence_diagram, dims, n_bins=100):
    points = persistence_diagram.get_finite_points()
    pi = PersistenceImage(sigma=0.1, n_bins=n_bins)
    pi_vector = []
    for dim in dims:
        dim_points = filter(lambda point: point.dim == dim, points)
        points_3d = np.array([list(map(lambda point: [point.birth, point.death, point.dim], dim_points))])
        if(points_3d.shape[1] == 0):
            pi_vector.append(np.zeros(shape=(n_bins,n_bins)))
        else:
            pi_vector.append(pi.fit_transform(points_3d)[0][0])
    return np.array(pi_vector).flatten()

In [26]:
def average_and_std_birth_and_death(points):
    if len(points) == 0:
        return [0, 0, 0, 0]
    births = np.array(list(map(lambda point: point.birth, points)))
    deaths = np.array(list(map(lambda point: point.death, points)))
    return [np.std(births), np.std(deaths), np.mean(births), np.mean(deaths)]

In [27]:
def get_average_birth_death_vector(persistence_diagram, dims):
    birth_eath_averages = []
    finite_points_pd = persistence_diagram.get_finite_points()
    for dim in dims:
        points_of_dim = list(filter(lambda point: point.dim == dim, finite_points_pd))
        birth_eath_averages.extend(average_and_std_birth_and_death(points_of_dim))
    birth_eath_averages.extend(average_birth_death(points_of_dim))
    return np.array(birth_eath_averages)

In [28]:
def get_average_birth_death_vector_squared(persistence_diagram, dims):
    birth_eath_averages = []
    finite_points_pd = persistence_diagram.get_finite_points()
    for dim in dims:
        points_of_dim = list(filter(lambda point: point.dim == dim, finite_points_pd))
        birth_eath_averages.extend(average_and_std_birth_and_death(points_of_dim))
    birth_eath_averages.extend(average_birth_death(points_of_dim))
    birth_eath_averages_squared = np.array(birth_eath_averages)
    return np.hstack([birth_eath_averages_squared, birth_eath_averages_squared**2])

In [29]:
def get_complex_roots(persistence_diagram, dims, n_coefficients=10, pol_type='T'):
    points = persistence_diagram.get_finite_points()
    complex_pol = ComplexPolynomial(n_coefficients=n_coefficients, polynomial_type=pol_type)
    complex_pol_vector = []
    for dim in dims:
        dim_points = filter(lambda point: point.dim == dim, points)
        points_3d = np.array([list(map(lambda point: [point.birth, point.death, point.dim], dim_points))])
        if(points_3d.shape[1] == 0):
            complex_pol_vector.extend([0]*(2*n_coefficients))
        else:
            complex_pol_vector.extend(complex_pol.fit_transform(points_3d)[0])
    return np.array(complex_pol_vector)

## Crear un dict con todas las features para todos los modelos de task 1 y task 2 y guardarlo en memoria para manipularlo, luego hacer estudios, que ya veremos cuales

In [30]:
from tqdm import tqdm

def bootstraping_function(X, pd_to_vec_fn, dims, samples_per_try=20, tries=5): #Change to 20, 5
    def bootstraping_elem(X_elem):
        vec_values_per_try = []
        for try_it in range(tries):
            try_sample = []
            for sample_it in range(samples_per_try):
                choice = random.randint(0, len(X_elem) - 1)
                try_sample.append(pd_to_vec_fn(X_elem[choice], dims))
            vec_values_per_try.append(np.mean(np.array(try_sample), axis=0))
        return np.mean(np.vstack(vec_values_per_try), axis=0)
    return list(map(bootstraping_elem, tqdm(X)))

In [33]:
def get_bootstraped_features(pds_x_gens, pd_to_vec_fn):
    working_dims=[('dim0', [0]), ('dim1', [1]), ('dim01',[0,1])]
    def get_pds_and_gen(pds_gen):
        _, gen_x_pds = list(zip(*pds_gen))
        return list(zip(*gen_x_pds))
    features_x_dims = dict()
    y, X_raw = get_pds_and_gen(pds_x_gens)
    for hashstr, dims in working_dims:
        # X_feature = bootstraping_function(X_raw, pd_to_vec_fn, dims)
        X_feature = list(map(lambda pd: pd_to_vec_fn(pd[0], dims), tqdm(X_raw)))
        features_x_dims[hashstr] = X_feature
    return y, features_x_dims

 ###  Per each feature and task we return a dictionary with the features computed at dims == key

In [34]:
y_values, pooling_task1 = get_bootstraped_features(pds_x_gen_task1, get_pooling_vector)
# y_values, pooling_task2 = get_bootstraped_features(pds_x_gen_task2, get_pooling_vector)

100%|██████████| 90/90 [00:03<00:00, 27.79it/s]


In [35]:
y_values, life_midlife_task1 = get_bootstraped_features(pds_x_gen_task1, get_life_midlife_vector)
#y_values, life_midlife_task2 = get_bootstraped_features(pds_x_gen_task2, get_life_midlife_vector)

100%|██████████| 90/90 [00:31<00:00,  2.82it/s]


In [36]:
y_values, life_midlife_squared_task1 = get_bootstraped_features(pds_x_gen_task1, get_life_midlife_vector_squared)
#y_values, life_midlife_squared_task2 = get_bootstraped_features(pds_x_gen_task2, get_life_midlife_vector_squared)

100%|██████████| 90/90 [00:32<00:00,  2.78it/s]


In [37]:
y_values, average_birth_death_task1 = get_bootstraped_features(pds_x_gen_task1, get_average_birth_death_vector)
#y_values, average_birth_death_task2 = get_bootstraped_features(pds_x_gen_task2, get_average_birth_death_vector)

100%|██████████| 90/90 [00:03<00:00, 26.75it/s]


In [38]:
y_values, average_birth_death_squared_task1 = get_bootstraped_features(pds_x_gen_task1, get_average_birth_death_vector_squared)
#y_values, average_birth_death_squared_task2 = get_bootstraped_features(pds_x_gen_task2, get_average_birth_death_vector_squared)

100%|██████████| 90/90 [00:04<00:00, 21.19it/s]


In [39]:
y_values, average_birth_death_inverted_task1 = get_bootstraped_features(pds_x_gen_task1, get_average_birth_death_vector_inverted)
#y_values, average_birth_death_inverted_task2 = get_bootstraped_features(pds_x_gen_task2, get_average_birth_death_vector_inverted)

100%|██████████| 90/90 [00:05<00:00, 15.89it/s]


In [40]:
y_values, average_birth_death_life_midlife_task1 = get_bootstraped_features(pds_x_gen_task1, get_average_birth_death_life_midlife_vector)
#y_values, average_birth_death_life_midlife_task2 = get_bootstraped_features(pds_x_gen_task2, get_average_birth_death_life_midlife_vector)

100%|██████████| 90/90 [00:32<00:00,  2.73it/s]


In [41]:
y_values, entropy_task1 = get_bootstraped_features(pds_x_gen_task1, get_entropy)
#y_values, entropy_task2 = get_bootstraped_features(pds_x_gen_task2, get_entropy)

100%|██████████| 90/90 [00:04<00:00, 18.35it/s]


In [42]:
y_values, number_of_points_task1 = get_bootstraped_features(pds_x_gen_task1, get_number_of_points)
#y_values, number_of_points_task2 = get_bootstraped_features(pds_x_gen_task2, get_number_of_points)

100%|██████████| 90/90 [00:04<00:00, 18.32it/s]


In [43]:
y_values, average_std_birth_death_task1 = get_bootstraped_features(pds_x_gen_task1, get_average_birth_death_vector)
#y_values, average_std_birth_death_task2 = get_bootstraped_features(pds_x_gen_task2, get_average_birth_death_vector)

100%|██████████| 90/90 [00:04<00:00, 19.00it/s]


In [44]:
y_values, average_std_birth_death_squared_task1 = get_bootstraped_features(pds_x_gen_task1, get_average_birth_death_vector_squared)
#y_values, average_std_birth_death_Squared_task2 = get_bootstraped_features(pds_x_gen_task2, get_average_birth_death_vector_squared)

100%|██████████| 90/90 [00:06<00:00, 14.98it/s]


In [45]:
y_values, complex_roots_task1 = get_bootstraped_features(pds_x_gen_task1, get_complex_roots)
#y_values, complex_roots_task2 = get_bootstraped_features(pds_x_gen_task2, get_complex_roots)

100%|██████████| 90/90 [03:19<00:00,  2.21s/it]


# Generating a pickle file

In [48]:
all_features_task1 = dict()
# all_features_task2 = dict()

In [49]:
all_features_task1['pooling'] = pooling_task1
# all_features_task2['pooling'] = pooling_task2

In [50]:
all_features_task1['life_midlife'] = life_midlife_task1
# all_features_task2['life_midlife'] = life_midlife_task2

In [51]:
all_features_task1['life_midlife_squared'] = life_midlife_squared_task1
#all_features_task2['life_midlife_squared'] = life_midlife_squared_task2

In [52]:
all_features_task1['average_birth_death'] = average_birth_death_task1
#all_features_task2['average_birth_death'] = average_birth_death_task2

In [53]:
all_features_task1['average_birth_death_squared'] = average_birth_death_squared_task1
#all_features_task2['average_birth_death_squared'] = average_birth_death_squared_task2

In [54]:
all_features_task1['average_birth_death_inverted'] = average_birth_death_inverted_task1
#all_features_task2['average_birth_death_inverted'] = average_birth_death_inverted_task2

In [55]:
all_features_task1['average_birth_death_life_midlife'] = average_birth_death_life_midlife_task1
#all_features_task2['average_birth_death_life_midlife'] = average_birth_death_life_midlife_task2

In [56]:
all_features_task1['entropy'] = entropy_task1
#all_features_task2['entropy'] = entropy_task2

In [57]:
all_features_task1['number_of_points'] = number_of_points_task1
#all_features_task2['number_of_points'] = number_of_points_task2

In [58]:
all_features_task1['average_std_birth_death'] = average_std_birth_death_task1
#all_features_task2['average_std_birth_death'] = average_std_birth_death_task2

In [59]:
all_features_task1['average_std_birth_death_squared'] = average_std_birth_death_squared_task1
#all_features_task2['average_std_birth_death_squared'] = average_std_birth_death_squared_task2

In [60]:
all_features_task1['complex_roots'] = complex_roots_task1
#all_features_task2['complex_roots'] = complex_roots_task2

**Saving the new dataset**

In [158]:

with open(outdir / '../../all_features_task1importancef2000.pickle', 'wb') as handle1:
    pickle.dump(all_features_task1, handle1, protocol=pickle.HIGHEST_PROTOCOL)

# with open('all_features_task2.pickle', 'wb') as handle2:
#     pickle.dump(all_features_task2, handle2, protocol=pickle.HIGHEST_PROTOCOL)


Now we can start working on mixed models

**Loading the new dataset**

In [43]:
with open(outdir / '../all_features_task1importancef2000.pickle', 'rb') as handle1:
    all_features_task1 = pickle.load(handle1)

# with open('all_features_task2.pickle', 'rb') as handle2:
#     all_features_task2 = pickle.load(handle2)

# Here, it starts the research

In [61]:
from itertools import combinations
from sklearn.model_selection import train_test_split

In [62]:
def classic_linear_model():
    return LinearRegression()
def ridge_linear_model():
    return linear_model.Ridge(alpha=.5)
def lasso_linear_model():
    return linear_model.Lasso(alpha=0.1)
def elasticNet():
    return ElasticNet()

Generate y train values

In [63]:
def get_y_values(pds_x_gens):
    def get_pds_and_gen(pds_gen):
        _, gen_x_pds = list(zip(*pds_gen))
        return list(zip(*gen_x_pds))
    features_x_dims = dict()
    y, _ = get_pds_and_gen(pds_x_gens)
    return y

In [64]:
y_task1 = get_y_values(pds_x_gen_task1)
# y_task2 = get_y_values(pds_x_gen_task2)

In [65]:
def get_all_possible_combinations_without_empty_set(all_features):
    possible_combinations = list()
    keys = all_features.keys()
    for cardinality in range(1, len(keys) + 1):
        possible_combinations.extend(combinations(keys, cardinality))
    return possible_combinations

In [79]:
combos = get_all_possible_combinations_without_empty_set(all_features_task1)

In [66]:
def generate_experiment(all_features, y, possible_dimensions=['dim0', 'dim1', 'dim01'], test_size=0.3, create_model_fn=classic_linear_model, number_of_experiments=10):
    results = dict()
    possible_combinations = get_all_possible_combinations_without_empty_set(all_features)
    for possible_combination in tqdm(possible_combinations):
        features_to_train = list()
        for feature in possible_combination:
            for possible_dimension in possible_dimensions:
                features_to_train.append(all_features[feature][possible_dimension])
        X = np.hstack(features_to_train)
        scores = list()
        for _ in range(number_of_experiments):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
            reg = create_model_fn().fit(X_train, y_train)
            scores.append(reg.score(X_test, y_test))
        results[tuple(possible_combination)] = np.mean(scores)
    return results

In [67]:
results_task1 = generate_experiment(all_features_task1, y_task1, create_model_fn=classic_linear_model)

100%|██████████| 4095/4095 [03:30<00:00, 19.41it/s]


In [166]:
# results_task2 = generate_experiment(all_features_task2, y_task2, create_model_fn=classic_linear_model)

In [70]:
import operator
max(results_task1.items(), key=operator.itemgetter(1))

(('average_birth_death',
  'average_birth_death_squared',
  'average_birth_death_life_midlife',
  'average_std_birth_death'),
 0.8935729120890972)

In [90]:
import operator
max(results_task2.items(), key=operator.itemgetter(1))

(('life_midlife_squared',
  'average_birth_death',
  'average_birth_death_squared',
  'entropy',
  'average_std_birth_death_squared'),
 0.9009350185767977)

In [22]:
def get_all_possible_combinations_without_empty_set_all(selected_features_keys, possible_dimensions=['dim0', 'dim1', 'dim01']):
    possible_combinations = list()
    keys = list()
    for key_raw in selected_features_keys:
        for possible_dim in possible_dimensions:
            keys.append((key_raw, possible_dim))
    for cardinality in range(1, len(keys) + 1):
        possible_combinations.extend(combinations(keys, cardinality))
    return possible_combinations

In [23]:
def generate_experiment_all(all_features, selected_features_keys, y, possible_dimensions=['dim0', 'dim1', 'dim01'], test_size=0.3, create_model_fn=classic_linear_model, number_of_experiments=10):
    results = dict()
    possible_combinations = get_all_possible_combinations_without_empty_set_all(selected_features_keys, possible_dimensions=possible_dimensions)
    for possible_combination in possible_combinations:
        features_to_train = list()
        for feature, dim in possible_combination:
            features_to_train.append(all_features[feature][dim])
        X = np.hstack(features_to_train)
        scores = list()
        try:
            for _ in range(number_of_experiments):
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
                reg = create_model_fn().fit(X_train, y_train)
                scores.append(reg.score(X_test, y_test))
            results[tuple(possible_combination)] = np.mean(scores)
            
        except:
            print('The next combination cannot be trained with a linear model')
            print(possible_combination)
    return results

In [36]:
results_task1 = generate_experiment_all(all_features_task1, (
    'life_midlife',
  'life_midlife_squared',
    'average_birth_death',
  'entropy',
    'average_std_birth_death',
  'average_std_birth_death_squared'), y_task1, create_model_fn=classic_linear_model)

In [37]:
results_task2 = generate_experiment_all(all_features_task2, (
    'life_midlife',
  'life_midlife_squared',
    'average_birth_death',
  'entropy',
    'average_std_birth_death',
  'average_std_birth_death_squared'), y_task2, create_model_fn=classic_linear_model)

In [38]:
import operator
max(results_task1.items(), key=operator.itemgetter(1))

((('life_midlife_squared', 'dim0'),
  ('average_std_birth_death', 'dim01'),
  ('average_std_birth_death_squared', 'dim0')),
 0.9243755781111016)

In [39]:
import operator
max(results_task2.items(), key=operator.itemgetter(1))

((('entropy', 'dim0'),
  ('entropy', 'dim1'),
  ('average_std_birth_death', 'dim01'),
  ('average_std_birth_death_squared', 'dim0')),
 0.9230907791163198)